In [2]:
from pathlib import Path
import typing as T
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy
import xarray as xr
import iris
import iris.plot
import numpy as np
import os
import sys
import textwrap
import itertools
import ants
import warnings

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/esmf.py:26: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/_ugrid.py:19: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/experimental/raster.py:29: IrisDeprecation: iris.experimental.raster is deprecated since version 3.2, and will be removed in a future release. If you make use of this functionality, please contact the Iris Developers to discuss how to retain it (which may involve reversing the deprecation).
  warn_deprecated(wmsg)


In [3]:
PATH = Path('/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um')

In [4]:
files = PATH.glob('*.nc')

In [5]:
for file in files:
    print (file)

/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverd012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20240920T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_p

In [5]:
files = PATH.glob('*.nc')
output = {}
long_names = []
var_names = []
for file in files:
    cubes = iris.load(file)
    for cube in cubes:
        #print (f'{file.name} {cube.long_name} {cube.standard_name} {cube.var_name}')
        long_names.append(cube.long_name)
        var_names.append(cube.var_name)
        output[cube.var_name] = { 'long name' : cube.long_name, 'file' : file.name}

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/__init__.py:326: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  return _load_collection(uris, constraints, callback).merged().cubes()
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/fileformats/_nc_load_rules/helpers.py:1179: _WarnComboDefaultingCfLoad: Failed to create 'VTACC1HR' dimension coordinate: The 'time' DimCoord bounds array must be strictly monotonic.
Gracefully creating 'VTACC1HR' auxiliary coordinate instead.
  warnings.warn(


In [6]:
unique_long_names = (list(set(long_names)))
print (unique_long_names)

['TURBULENT MIXING HT AFTER B.LAYER m', 'OUTGOING LW RAD FLUX (TOA)', 'NET DOWN SURFACE LW RAD FLUX', 'NET DOWN SURFACE SW FLUX : CORRECTED', 'TEMPERATURE AT 1.5M', 'RELATIVE HUMIDITY AT 1.5M', 'OUTGOING SW RAD FLUX (TOA)', 'OUTGOING SW RAD FLUX (TOA):CORRECTED', 'TOTAL COLUMN QCL  RHO GRID', 'NUMBER OF LIGHTNING FLASHES', 'LAND MASK (No halo) (LAND=TRUE)', 'CLD ICE MIXING RATIO (mcf) AFTER TS', 'TOTAL CLOUD AMOUNT MAX/RANDOM OVERLP', 'LARGE SCALE RAIN AMOUNT     KG/M2/TS', 'HIGH CLOUD AMOUNT', 'PRESSURE AT MEAN SEA LEVEL', 'WIND GUST  (M/S)', 'V WIND ON PRESSURE LEVELS    B GRID', 'GRAUPEL NUMBER AFTER TIMESTEP', 'VIS AT 1.5M  (incl precip)         M', '10 METRE WIND V-COMP         B GRID', 'TOTAL DOWNWARD SURFACE SW FLUX', 'TOTAL COLUMN DRY MASS  RHO GRID', 'OROGRAPHY (/STRAT LOWER BC)', 'DEWPOINT AT 1.5M (K)', 'GEOPOTENTIAL HEIGHT ON P LEV/P GRID', 'MEDIUM CLOUD AMOUNT', 'LARGE SCALE SNOWFALL RATE    KG/M2/S', 'W ON GEOMETRIC HEIGHT LEVELS', 'CUMULUS-CAPPED BL INDICATOR', 'RH WRT IC

In [7]:
len(unique_long_names)

60

In [8]:
unique_var_names = (list(set(var_names)))
print (unique_var_names)

['STASH_m01s04i202', 'STASH_m01s16i203_2', 'STASH_m01s03i281_2', 'STASH_m01s03i217', 'STASH_m01s08i225_2', 'STASH_m01s02i207_2', 'STASH_m01s30i406_2', 'STASH_m01s03i304', 'STASH_m01s09i202_2', 'STASH_m01s09i205_3', 'STASH_m01s03i463', 'STASH_m01s16i203', 'STASH_m01s09i204', 'STASH_m01s04i203', 'STASH_m01s02i205', 'STASH_m01s02i205_2', 'STASH_m01s01i235_2', 'STASH_m01s00i393', 'STASH_m01s03i236_2', 'STASH_m01s09i203', 'STASH_m01s09i204_3', 'STASH_m01s08i223_2', 'STASH_m01s00i409', 'STASH_m01s30i403_2', 'STASH_m01s00i024', 'STASH_m01s30i406', 'STASH_m01s03i225', 'STASH_m01s00i031', 'STASH_m01s16i222', 'STASH_m01s09i202', 'STASH_m01s09i203_3', 'STASH_m01s04i112_2', 'STASH_m01s00i023', 'STASH_m01s15i202', 'STASH_m01s15i201', 'STASH_m01s30i404', 'STASH_m01s04i203_2', 'STASH_m01s03i226_2', 'STASH_m01s16i204_2', 'STASH_m01s16i204', 'STASH_m01s00i081', 'STASH_m01s04i112', 'STASH_m01s30i405_2', 'STASH_m01s03i234', 'STASH_m01s16i205_2', 'STASH_m01s04i201_2', 'STASH_m01s15i142', 'STASH_m01s21i104

In [9]:
len(unique_var_names)

109

In [10]:
# Compare to STASHC file
STASHC_file='/home/548/pag548/cylc-run/u-dq126/work/20220227T0000Z/Flagship_ERA5to1km_5km_RAL3P2_um_fcst_000/STASHC'

In [11]:
with open(STASHC_file, "r") as file:
    stash = file.read()

In [12]:
stash.count('&umstash_streq')

132

In [13]:
stash.count('item=')

132

In [14]:
# Ok let's look at the levels of various files.
# TEMPERATURE ON P LEV/P GRID air_temperature STASH_m01s16i203_2

In [15]:
stash[:500]

"&umstash_streq\ndom_name='DIAG',\nisec=4,\nitem=201,\npackage='',\ntim_name='t1min',\nuse_name='USR_DIAGS',\n/\n&umstash_streq\ndom_name='DIAG',\nisec=5,\nitem=201,\npackage='',\ntim_name='tsmin',\nuse_name='USR_DIAGS',\n/\n&umstash_streq\ndom_name='DIAG',\nisec=16,\nitem=222,\npackage='',\ntim_name='t1min',\nuse_name='USR_DIAGS',\n/\n&umstash_streq\ndom_name='DIAG',\nisec=4,\nitem=201,\npackage='',\ntim_name='tsmin',\nuse_name='USR_DIAGS',\n/\n&umstash_streq\ndom_name='DIAG',\nisec=16,\nitem=222,\npackage='',\ntim_name='tsmin',\nus"

In [6]:
cubes = iris.load(PATH / 'umnsaa_pverd000.nc')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/__init__.py:326: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  return _load_collection(uris, constraints, callback).merged().cubes()


In [7]:
cubes

Air Temperature (K),time,air_pressure,grid_latitude,grid_longitude
Shape,1,16,976,1466
Dimension coordinates,,,,
time,x,-,-,-
air_pressure,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
latitude,-,-,x,x
longitude,-,-,x,x
Cell methods,,,,


In [12]:
temp = cubes[1]
temp

Air Temperature (K),time,air_pressure,grid_latitude,grid_longitude
Shape,4,16,976,1466
Dimension coordinates,,,,
time,x,-,-,-
air_pressure,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
latitude,-,-,x,x
longitude,-,-,x,x
Cell methods,,,,


In [13]:
temp.coord('air_pressure').points

array([1000.,  950.,  925.,  850.,  800.,  750.,  700.,  650.,  600.,
        500.,  400.,  300.,  250.,  200.,  150.,  100.])

In [14]:
temp.coord('time').points

array([ 3.,  6.,  9., 12.])

In [15]:
temp.coord('time').units

Unit('hours since 2024-09-20 00:00:00', calendar='standard')

In [22]:
temp.attributes['STASH']

STASH(model=1, section=16, item=203)

There are only two entries in STASHC with this configuration

In [23]:
for st in unique_var_names:
    if 'STASH_m01s16i203' in st:
        print (st,output[st])

STASH_m01s16i203_2 {'long name': 'TEMPERATURE ON P LEV/P GRID', 'file': 'umnsaa_pverd000.nc'}
STASH_m01s16i203 {'long name': 'TEMPERATURE ON P LEV/P GRID', 'file': 'umnsaa_pverd000.nc'}


In [24]:
# Read the second temperature cube
temp_2=cubes[1]
temp_2.coord('time').points

array([0.])

In [25]:
temp_2.coord('time').units

Unit('days since 2022-02-27 00:00:00', calendar='standard')

In [26]:
# Update - see if graupel is in this file
PATH_UPDATE = Path('/home/548/pag548/cylc-run/u-dq126/share/cycle/20220226T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um')

In [27]:
files = PATH_UPDATE.glob('*.nc')
output_update = {}
long_names_update = []
var_names_update = []
for file in files:
    cubes = iris.load(file)
    for cube in cubes:
        #print (f'{file.name} {cube.long_name} {cube.standard_name} {cube.var_name}')
        long_names_update.append(cube.long_name)
        var_names_update.append(cube.var_name)
        output[cube.var_name] = { 'long name' : cube.long_name, 'file' : file.name}

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/fileformats/cf.py:1222: IrisCfNonSpanningVarWarning: Ignoring variable 'TH_1_80_model_level_number' referenced by variable 'STASH_m01s00i010': Dimensions ('TH_1_80_eta_rho',) do not span ('T1HR', 'TH_1_80_eta_theta', 'grid_latitude_t', 'grid_longitude_t')
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/fileformats/cf.py:1222: IrisCfNonSpanningVarWarning: Ignoring variable 'TH_1_80_model_level_number' referenced by variable 'STASH_m01s03i513': Dimensions ('TH_1_80_eta_rho',) do not span ('T1HR_MN', 'TH_1_80_eta_theta', 'grid_latitude_t', 'grid_longitude_t')
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/fileformats/cf.py:1222: IrisCfNonSpanningVarWarning: Ignoring variable 'TH_1_80_model_level_number' referenced by variable 'STASH_m01s16i004': Dimensions ('TH_1_80_eta_rho',) do not

In [28]:
unique_long_names_update = (list(set(long_names_update)))
print (unique_long_names_update)

['TURBULENT MIXING HT AFTER B.LAYER m', 'TOTAL RADAR REFLECTIVITY 3D  (dBZ)', 'TEMPERATURE AT 1.5M', 'OUTGOING SW RAD FLUX (TOA)', 'TOTAL COLUMN QCL  RHO GRID', 'NUMBER OF LIGHTNING FLASHES', 'LAND MASK (No halo) (LAND=TRUE)', '10 METRE WIND V-COMP         B GRID', 'OROGRAPHY (/STRAT LOWER BC)', 'CLEAR-SKY (II) DOWN SURFACE LW FLUX', 'SOIL MOISTURE CONTENT IN A LAYER', 'TEMPERATURE ON THETA LEVELS', 'OMEGA ON THETA LEVELS (C GRID)', 'SBCAPE surface based CAPE       J/kg', 'SUPERCOOLED LIQUID WATER CONTENT', 'TOTAL COLUMN WET MASS  RHO GRID', 'ICE NUMBER AFTER TIMESTEP', 'WET BULB POTENTIAL TEMPERATURE     K', 'LARGE SCALE SNOW AMOUNT     KG/M2/TS', '10 METRE WIND U-COMP         B GRID', 'RAIN NUMBER AFTER TIMESTEP', 'CLEAR-SKY (II) UPWARD LW FLUX (TOA)', 'BOUNDARY LAYER DEPTH AFTER TIMESTEP', 'OUTGOING LW RAD FLUX (TOA)', 'GRAUPEL AFTER TIMESTEP', 'SURFACE DOWNWARD LW RADIATION   W/M2', 'HIGH CLOUD AMOUNT', 'WIND GUST  (M/S)', 'GRAUPEL NUMBER AFTER TIMESTEP', 'VIS AT 1.5M  (incl precip

In [29]:
len(unique_long_names_update)

97

In [30]:
unique_var_names_update = (list(set(var_names_update)))
print (unique_var_names_update)

['STASH_m01s04i202', 'STASH_m01s00i003', 'STASH_m01s16i004', 'STASH_m01s00i408', 'STASH_m01s16i203_2', 'STASH_m01s03i281_2', 'STASH_m01s03i217', 'STASH_m01s04i118', 'STASH_m01s00i392', 'STASH_m01s08i225_2', 'STASH_m01s02i207_2', 'STASH_m01s30i406_2', 'STASH_m01s00i273', 'STASH_m01s00i394', 'STASH_m01s03i304', 'STASH_m01s00i012', 'STASH_m01s09i202_2', 'STASH_m01s09i205_3', 'TH_1_80_model_level_number', 'STASH_m01s00i254', 'STASH_m01s01i209', 'STASH_m01s03i463', 'STASH_m01s00i075', 'STASH_m01s16i203', 'STASH_m01s09i204', 'STASH_m01s04i203', 'STASH_m01s00i079', 'STASH_m01s02i205', 'STASH_m01s01i207', 'STASH_m01s00i271', 'STASH_m01s01i210', 'STASH_m01s02i205_2', 'STASH_m01s01i235_2', 'STASH_m01s00i393', 'STASH_m01s03i236_2', 'STASH_m01s09i216', 'STASH_m01s09i203', 'STASH_m01s09i204_3', 'STASH_m01s08i223_2', 'STASH_m01s00i409', 'STASH_m01s30i403_2', 'STASH_m01s00i024', 'STASH_m01s30i406', 'STASH_m01s03i225', 'STASH_m01s00i031', 'STASH_m01s16i222', 'STASH_m01s09i202', 'STASH_m01s09i203_3', '

In [31]:
len(unique_var_names_update)

147

In [39]:
# Diff b/w unique long_names and unique_long_names_update
list(set(unique_long_names)-set(unique_long_names_update))

[]

In [33]:
unique_long_names

['TURBULENT MIXING HT AFTER B.LAYER m',
 'OUTGOING LW RAD FLUX (TOA)',
 'NET DOWN SURFACE LW RAD FLUX',
 'NET DOWN SURFACE SW FLUX : CORRECTED',
 'TEMPERATURE AT 1.5M',
 'RELATIVE HUMIDITY AT 1.5M',
 'OUTGOING SW RAD FLUX (TOA)',
 'OUTGOING SW RAD FLUX (TOA):CORRECTED',
 'TOTAL COLUMN QCL  RHO GRID',
 'NUMBER OF LIGHTNING FLASHES',
 'LAND MASK (No halo) (LAND=TRUE)',
 'CLD ICE MIXING RATIO (mcf) AFTER TS',
 'TOTAL CLOUD AMOUNT MAX/RANDOM OVERLP',
 'LARGE SCALE RAIN AMOUNT     KG/M2/TS',
 'HIGH CLOUD AMOUNT',
 'PRESSURE AT MEAN SEA LEVEL',
 'WIND GUST  (M/S)',
 'V WIND ON PRESSURE LEVELS    B GRID',
 'GRAUPEL NUMBER AFTER TIMESTEP',
 'VIS AT 1.5M  (incl precip)         M',
 '10 METRE WIND V-COMP         B GRID',
 'TOTAL DOWNWARD SURFACE SW FLUX',
 'TOTAL COLUMN DRY MASS  RHO GRID',
 'OROGRAPHY (/STRAT LOWER BC)',
 'DEWPOINT AT 1.5M (K)',
 'GEOPOTENTIAL HEIGHT ON P LEV/P GRID',
 'MEDIUM CLOUD AMOUNT',
 'LARGE SCALE SNOWFALL RATE    KG/M2/S',
 'W ON GEOMETRIC HEIGHT LEVELS',
 'CUMULUS-CAP

In [34]:
unique_long_names_update

['TURBULENT MIXING HT AFTER B.LAYER m',
 'TOTAL RADAR REFLECTIVITY 3D  (dBZ)',
 'TEMPERATURE AT 1.5M',
 'OUTGOING SW RAD FLUX (TOA)',
 'TOTAL COLUMN QCL  RHO GRID',
 'NUMBER OF LIGHTNING FLASHES',
 'LAND MASK (No halo) (LAND=TRUE)',
 '10 METRE WIND V-COMP         B GRID',
 'OROGRAPHY (/STRAT LOWER BC)',
 'CLEAR-SKY (II) DOWN SURFACE LW FLUX',
 'SOIL MOISTURE CONTENT IN A LAYER',
 'TEMPERATURE ON THETA LEVELS',
 'OMEGA ON THETA LEVELS (C GRID)',
 'SBCAPE surface based CAPE       J/kg',
 'SUPERCOOLED LIQUID WATER CONTENT',
 'TOTAL COLUMN WET MASS  RHO GRID',
 'ICE NUMBER AFTER TIMESTEP',
 'WET BULB POTENTIAL TEMPERATURE     K',
 'LARGE SCALE SNOW AMOUNT     KG/M2/TS',
 '10 METRE WIND U-COMP         B GRID',
 'RAIN NUMBER AFTER TIMESTEP',
 'CLEAR-SKY (II) UPWARD LW FLUX (TOA)',
 'BOUNDARY LAYER DEPTH AFTER TIMESTEP',
 'OUTGOING LW RAD FLUX (TOA)',
 'GRAUPEL AFTER TIMESTEP',
 'SURFACE DOWNWARD LW RADIATION   W/M2',
 'HIGH CLOUD AMOUNT',
 'WIND GUST  (M/S)',
 'GRAUPEL NUMBER AFTER TIMESTEP'

In [35]:
# Diff b/w unique var_names and unique_var_names_update
list(set(unique_var_names)-set(unique_var_names_update))

[]

In [36]:
[ print (s) for s in unique_long_names_update if 'GRAUPEL' in s ]

GRAUPEL AFTER TIMESTEP
GRAUPEL NUMBER AFTER TIMESTEP
GRAUPEL MIXING RATIO (mg) AFTER TS


[None, None, None]

In [37]:
for st in unique_var_names_update:
    if 'STASH_m01s00i012' in st:
        print (st,output[st])

STASH_m01s00i012 {'long name': 'QCF AFTER TIMESTEP', 'file': 'umnsaa_pa000.nc'}


In [38]:
unique_var_names_update = (list(set(var_names_update)))
print (unique_var_names_update)

['STASH_m01s04i202', 'STASH_m01s00i003', 'STASH_m01s16i004', 'STASH_m01s00i408', 'STASH_m01s16i203_2', 'STASH_m01s03i281_2', 'STASH_m01s03i217', 'STASH_m01s04i118', 'STASH_m01s00i392', 'STASH_m01s08i225_2', 'STASH_m01s02i207_2', 'STASH_m01s30i406_2', 'STASH_m01s00i273', 'STASH_m01s00i394', 'STASH_m01s03i304', 'STASH_m01s00i012', 'STASH_m01s09i202_2', 'STASH_m01s09i205_3', 'TH_1_80_model_level_number', 'STASH_m01s00i254', 'STASH_m01s01i209', 'STASH_m01s03i463', 'STASH_m01s00i075', 'STASH_m01s16i203', 'STASH_m01s09i204', 'STASH_m01s04i203', 'STASH_m01s00i079', 'STASH_m01s02i205', 'STASH_m01s01i207', 'STASH_m01s00i271', 'STASH_m01s01i210', 'STASH_m01s02i205_2', 'STASH_m01s01i235_2', 'STASH_m01s00i393', 'STASH_m01s03i236_2', 'STASH_m01s09i216', 'STASH_m01s09i203', 'STASH_m01s09i204_3', 'STASH_m01s08i223_2', 'STASH_m01s00i409', 'STASH_m01s30i403_2', 'STASH_m01s00i024', 'STASH_m01s30i406', 'STASH_m01s03i225', 'STASH_m01s00i031', 'STASH_m01s16i222', 'STASH_m01s09i202', 'STASH_m01s09i203_3', '